In [ ]:
!pip install kaggle
import kaggle

In [ ]:
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

In [ ]:
#Extracting data from zipped folder
import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip') #provide file path
zip_ref.extractall() #extract file to dir
zip_ref.close() #close file

In [ ]:
#importing the csv file
import pandas as pd
df = pd.read_csv("orders.csv")
df.head(20)

In [ ]:
#lets go through our data
df.info()

In [ ]:
#Though here there are no null values in my data, let's go through the column values to verify it
df['Order Id'].nunique() #For ID we can check unique count instead of fetching unique values

In [ ]:
df['Order Date'].unique()

In [ ]:
df['Ship Mode'].unique()

In [ ]:
#Here we have undesirable values which we will first convert to null
import numpy as np
df['Ship Mode'] = df['Ship Mode'].replace(['Not Available', 'unknown'],np.nan)
df['Ship Mode'].unique()

In [ ]:
df['Category'].unique()

In [ ]:
df.columns = df.columns.str.lower() #we will rename the columns by replacing space with underscore and converting to lower case for ease of use

In [ ]:
df.columns = df.columns.str.replace(' ','_')
df.head()

In [ ]:
#Let us add new columns for sale_price and profit
#discount = list_price - discount_percent
#sales_price = list_price - discount
#profit = cost_price - sales_price
df['discount'] = df['list_price']*df['discount_percent']*0.01
df['sales_price'] = df['list_price']-df['discount']
df['profit'] = df['list_price'] - df['sales_price']
df

In [ ]:
#Now we will remove columns that we don't need
df.drop(columns = ['list_price','discount_percent','cost_price'], inplace = True)

In [ ]:
#We will also convert the order_date column into datetime datatype for using it in SQL
df['order_date'] = pd.to_datetime(df['order_date'],format = "%Y-%m-%d")

In [ ]:
df.dtypes

In [ ]:
#Now we will load this data into SQL server for which first we will create connection
import sqlalchemy as sal
engine = sal.create_engine('mssql://LAPTOP-VO7RFI9I/Sales_Analysis?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
#Since I am using SSMS hence mssql
# Here LAPTOP-VO7RFI9I is my server name
#Sales_Analysis is my database name
#ODBC+DRIVER+17+FOR+SQL+SERVER is my driver name which I can get from ODBC application
conn=engine.connect()

In [ ]:
# Now we will add the dataframe in an existing database as a table
df.to_sql('df_orders', con = conn, index = False, if_exists = 'replace')
#df_orders will be my table name in SQL
#We don't want index column of dataframe in our SQL table hence index = False
#If table already exists then replace it
#The drawback of using 'replace' is that the datatypes in SQL get assigned to the max value automatically
#Instead we can use create the table in SQL by observing the data in Jupyter and then use if_exists = 'append' to add data to the table

### Create table script
CREATE TABLE df_orders(
	[order_id] int PRIMARY KEY,
	[order_date] date,
	[ship_mode] varchar(20),
	[segment] varchar(20),
	[country] varchar(20),
	[city] varchar(20),
	[state] varchar(20),
	[postal_code] varchar(20),
	[region] varchar(20),
	[category] varchar(20),
	[sub_category] varchar(20),
	[product_id] varchar(50),
	[quantity] int,
	[discount] decimal(7,2),
	[sales_price] decimal(7,2),
	[profit] decimal(7,2))

In [ ]:
#Appending data in existing table in SQL
df.to_sql('df_orders', con = conn, index = False, if_exists = 'append')